# Problem Set 3: Merging and regular expressions

**Total points (without extra credit)**: 44

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset3_inputdata` (need to unzip): 

- `jobs`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


## Resources from class

- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/06_qss20_w23_mergereshape.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/03_reshaping_merging_solutions.ipynb) on exact merging
- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/07_qss20_w23_regex.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/04_regex_blank.ipynb) on regular expressions (both coming in class on Feb. 1)
- DataCamp modules on both of these

# 0. Load packages & data (1 point total)

In [80]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 0.1 Load data (0 points)

Load the following dataset stored in `pset3_inputdata`&mdash;`debar.csv`&mdash;storing it as a dataframe named `debar`. This represents employers temporarily banned from hiring workers.

View the `head()` and columns of this dataframe.

In [81]:
## your code here
debar = pd.read_csv("../pset3_inputdata/debar.csv", low_memory=False)
debar.head()
debar.columns

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


Index(['Name', 'City, State', 'Violation', 'Duration', 'Start date',
       'End date'],
      dtype='object')

The most notable column names mean as follows:
- `Name`:	Company name of agricultural employer
- `City, State`:	City and state where employer located
- `Violation`:	Type of program violation
- `Start date`:	Start date of debarment (temporary ban)

## 0.2 Inspect data (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [82]:
## your code here
len(debar.index)
debar["Name"].nunique()

# 114 rows vs. 98
# some duplicate rows per employers

114

98

# 1. Reshape data and check duplicates (19 points total)

## 1.1 Make indicator for violation number (2 points)

To make it possible to reshape data, make an indicator for the violation number for each business. The indicator should take the value of `viol` if it's the first row/potential violation, `viol2` if the second row/potential violation, etc.

**Hint:**
- One way to do this is by using an if-else statement to check whether the business name is the same as in the row above (assuming rows are ordered by name). Grouping by employer name and checking the number of unique offenses would also work.

In [83]:
## your code here
debar["v_number"] = debar.groupby("Name").cumcount()
debar["v_number"] = debar["v_number"].apply(lambda x: f"viol{x + 1}")
debar["v_number"].replace("viol1", "viol", inplace=True)

## 1.2 Clean up state names (3 points)

Inspect the state names in the business violation data. Which states have names sometimes indicated in long format vs. two-letter abbreviation, e.g. "New Hampshire" vs. "NH"? Which of these may have more than one violation?

**Hint:** 
- One way to do this is to extract state names from the `City, State` column using regex and/or string methods.
- The simplest way to check if a state may have more than one violation is to check how many times they appear using `value_counts()`. 

In [84]:
## your code here
debar["State"] = debar["City, State"].str.split(", ").str[1]
debar["State"].value_counts()
# Texas, Georgia, and Florida are among a few occurences that are indicated in long and abbreviated format.
# All states with more than one occurence could have more than one violation.

TX                14
GA                10
KS                 9
Georgia            9
FL                 8
KY                 5
SD                 5
UT                 4
Texas              4
ND                 3
NY                 3
North Dakota       3
CA                 3
CO                 3
Florida            2
AR                 2
Utah               2
OK                 2
LA                 2
MA                 2
MT                 2
Tennessee          1
Idaho              1
North Carolina     1
Massachusetts      1
Louisiana          1
IL                 1
MD                 1
MN                 1
Vermont            1
Arkansas           1
California         1
SC                 1
ID                 1
Kansas             1
AK                 1
ME                 1
Name: State, dtype: int64

To make the state names consistent with the jobs data below, convert any discrepant state names to the two-letter abbreviation format. 

**Hints:**
- You could do this for all state names (to be safe) or only those you just identified. 
- If you want to change ANY discrepant state names from long format to two-letter format (i.e., a complete conversion), you can load state names from a complete name/abbreviation crosswalk and use that to change state names. Here is code to load in such a list ([from this blog](https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c)):
```python
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
pd.read_html(crosswalk_url)[0]
```

In [85]:
## your code here
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
table = pd.read_html(crosswalk_url)[0]
crosswalk = table.set_index(table.columns[1]).to_dict()[table.columns[0]]
debar["State1"] = debar["State"].map(crosswalk)

debar.loc[~debar["State1"].isna(), "State"] = debar.loc[~debar["State1"].isna(), "State1"]
debar = debar.drop(columns=["State1"])



debar["State"].value_counts()
# matches with above (GA 19 = GA 10 + Georgia 9)


GA    19
TX    18
KS    10
FL    10
ND     6
UT     6
SD     5
KY     5
CA     4
AR     3
NY     3
MA     3
CO     3
LA     3
MT     2
ID     2
OK     2
MN     1
SC     1
VT     1
TN     1
AK     1
MD     1
NC     1
IL     1
ME     1
Name: State, dtype: int64

## 1.3 Investigate duplicated rows (4 points)

A. Create a new column in `debar`, `is_repeated`, that tells us whether an employer (`Name`) is repeated > 1 times.

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using `value_counts()` on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret with at least a sentence. If you notice any cases of duplicate business names where 'City, State' does NOT match exactly, check these manually and make them consistent.

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [86]:
## your code here
# A.
debar["is_repeated"] = debar.groupby("Name")["Name"].transform("count") > 1


# B.
debar[debar["is_repeated"] == True].sort_values(by="Name")
# It seems that secondary violations are generally caused by a Failure to respond or impeding an 
# audit process by not responding or responding partially.

# returns the "City, State" var to the corrected version by concatenating the two parts
debar["City"] = debar["City, State"].str.split(",").str[0]
def concat_columns(row, City, State):
    return str(row[City]) + ", " + str(row[State])
debar["City, State"] = debar.apply(lambda row: concat_columns(row, "City", "State"), axis=1)

# cleaning typos in names
debar['City, State'] = np.where(debar['City, State']=='Altheimer, AK', 'Altheimer, AR', debar['City, State'])
debar['City, State'] = np.where(debar['City, State']=='Brownsville, TX', 'Brownfield, TX', debar['City, State'])

# show that typos are fixed
debar[debar['City, State'].str.contains(r'Brown|Althe', regex = True)]


# C.
# make a copy to avoid SetWithCopyWarnings
mult_debar = debar[debar["is_repeated"] == True].copy()
mult_debar.head()
mult_debar.shape





,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,UT,True
30,Annabella Land & Cattle,"Annabella, Utah",Non-payment,1 year,5/9/2014,5/8/2015,viol2,UT,True
29,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,2 years,7/6/2014,7/5/2016,viol2,MA,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,MA,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,TX,True
28,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, Texas",Impeding the Audit Process – Partial- Response,2 years,7/20/2014,7/19/2016,viol2,TX,True
56,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,viol2,GA,True
19,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,viol,GA,True
103,Dove Creek Farms,"Mount Vernon, TX",Failure to respond to audit request,2 years,2/9/2018,2/8/2018,viol,TX,True
109,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2 years,2/9/2018,2/8/2020,viol2,TX,True


,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,is_repeated,City
11,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol,TX,True,Brownsville
17,"Maple Ridge Custom Services, LLC","Altheimer, AR",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,viol,AK,True,Altheimer
25,Loewen Harvesting LLC,"Brownfield, TX",Impeding the Audit Process – Partial- Response,1 year,8/20/2014,8/19/2015,viol2,TX,True,Brownfield
59,"Maple Ridge Custom Services, LLC","Altheimer, AR",Impeding the Audit Process – Partial- Response,1 year,11/16/2014,11/15/2015,viol2,AR,True,Altheimer


,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,is_repeated,City
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,UT,True,Annabella
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,MA,True,Groton
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,TX,True,Caddo
11,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol,TX,True,Brownsville
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,viol,FL,True,Miami


(32, 10)

## 1.4 Reshape `mult_debar` to wide format to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in these data: e.g., converting `Violation` to lowercase or replacing spelled-out states with two-dig state codes.

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

For a successful reshaping, make sure each row in `mult_debar` shows only a single business (no duplicates under `Name`) and isn't missing either `start_date_viol1` or `start_date_viol2`.

Show the contents of `mult_debar` and its shape.

In [87]:
## your code here
# mult_d = mult_debar.drop_duplicates(subset=["Name", "City, State", "Start date"])
mult_debar_wide = mult_debar.pivot(index=['Name', 'City, State'], columns='v_number', values='Start date')
mult_debar_wide.rename(columns={'viol': 'start_date_viol1',
                                'viol2': 'start_date_viol2'},
                      inplace=True)
mult_debar_wide
mult_debar_wide.shape
# 16 is half of the 32 original rows --> sample got widened correctly


,v_number,start_date_viol1,start_date_viol2
Name,"City, State",,
Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014
Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014
Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015
Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018
F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014
Loewen Harvesting LLC,"Brownfield, TX",8/20/2014,8/20/2014
Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015
"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,11/16/2014


(16, 2)

## 1.5 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning, `debar`, filter as follows:
- For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
- For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
- Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [88]:
## your code here
# A. 
mult_debar_wide['is_dup'] = mult_debar_wide['start_date_viol1'] == mult_debar_wide['start_date_viol2']
mult_debar_wide = mult_debar_wide.reset_index()

# B. 
duplicate = mult_debar_wide[mult_debar_wide["is_dup"] == True]
non_dupe = mult_debar_wide[mult_debar_wide["is_dup"] == False]
duplicate_list = list(duplicate.Name)
ndl = list(non_dupe.Name)
duplicate_column = mult_debar_wide[mult_debar_wide["is_dup"] == True]["Name"]
subset = debar[debar["Name"].isin(duplicate_list)]
debar_clean = debar[~(debar["Name"].isin(duplicate_column) & (debar["v_number"] != "viol"))]
debar_clean = debar_clean.drop("is_repeated", axis=1)

# show that no duplicates remain doubled
debar_clean[debar_clean["Name"].isin(duplicate_column)]
# show that the non-duplicates remain doubled
debar_clean[debar_clean.Name.isin(ndl)].sort_values("Name")

debar_clean.shape
debar_clean["Name"].nunique()



,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,City
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol,UT,Annabella
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,MA,Groton
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol,TX,Caddo
11,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,viol,TX,Brownsville
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,viol,FL,Miami
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,GA,Tifton
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,TX,Morton
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,viol,UT,Roosevelt
17,"Maple Ridge Custom Services, LLC","Altheimer, AR",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,viol,AK,Altheimer
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,viol,KS,Ingalls


,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,City
19,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,viol,GA,Cairo
56,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,viol2,GA,Cairo
21,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",WHD Debarment,3 years,12/11/2014,12/10/2017,viol,SD,Volga
51,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",Wage Hour Debarment,3 years,12/1/2014,12/1/2017,viol2,SD,Volga
73,Xavier Horne,"Lyons, GA",Non-payment of certification fee,1 year,6/16/2016,6/15/2017,viol,GA,Lyons
89,Xavier Horne,"Lyons, GA",Failure to respond to audit request,2 years,9/27/2017,9/26/2019,viol2,GA,Lyons


(101, 9)

98

# 2. Merging and regex (19 points total)



## 2.1 Load data on job postings (1 point)

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations. Since most of the bans have expired, let's see which of those employers posted new H-2A jobs in the first quarter of 2021.

First, load the `jobs.csv` data stored in `pset3_inputdata`.

In [89]:
# load data
jobs = pd.read_csv("../pset3_inputdata/jobs.csv", low_memory=False)
jobs.head()
jobs.columns

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


Index(['CASE_NUMBER', 'CASE_STATUS', 'RECEIVED_DATE', 'DECISION_DATE',
       'TYPE_OF_EMPLOYER_APPLICATION', 'H2A_LABOR_CONTRACTOR',
       'NATURE_OF_TEMPORARY_NEED', 'EMERGENCY_FILING', 'EMPLOYER_NAME',
       'TRADE_NAME_DBA',
       ...
       'ADDENDUM_B_HOUSING_ATTACHED', 'TOTAL_HOUSING_RECORDS',
       'MEALS_PROVIDED', 'MEALS_CHARGED', 'MEAL_REIMBURSEMENT_MINIMUM',
       'MEAL_REIMBURSEMENT_MAXIMUM', 'PHONE_TO_APPLY', 'EMAIL_TO_APPLY',
       'WEBSITE_TO_APPLY', 'TOTAL_ADDENDUM_A_RECORDS'],
      dtype='object', length=138)

The most notable column names mean as follows:
- `CASE_NUMBER`:	Administrative identifier for an employer's H-2A visa application
- `EMPLOYER_NAME`:	Employer name
- `EMPLOYER_CITY`:	Employer city
- `EMPLOYER_STATE`:	Employer state
- `EMPLOYER_ADDRESS_1`:	Employer address (only need to use if doing the geocoding extra credit)

##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [90]:
## your code here
joined = pd.merge(jobs, debar_clean,
                  left_on='EMPLOYER_NAME', right_on='Name',
                  how='inner')
joined
# only one exact match found

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,City
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,7,Rafael Barajas,"Sebring, FL",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol,FL,Sebring


## 2.3 Targeted regex (11 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Convert to upper (3 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [91]:
## insert your code to turn into uppercase here
jobs["EMPLOYER_NAME"] = [name.upper() for name in jobs["EMPLOYER_NAME"]]
debar_clean["Name"] = [name.upper() for name in debar_clean["Name"]]

In [92]:
## insert your code for the random sample
jobs["EMPLOYER_NAME"].sample(n=15)
debar_clean["Name"].sample(n=15)

1614                SODVILLE FARM SERVICES, INC.
1086                               SAHA FISH PTN
2355                  KENNETH ROSS OLIVIER FARMS
2496                   CHACON SHEEP COMPANY, LLC
2100                       AMERICAN CRAWFISH LLC
2028                  MAPLE RIDGE FRUIT FARM LLC
2053                               CALHOON RANCH
1272                   WESTERN RANGE ASSOCIATION
2460                            CARONA FARMS LLC
674                        CANTON MART FARMS PTR
55                     WESTERN RANGE ASSOCIATION
2636                         HERMANSDALE FARM GP
1399                               BRIAN MCNANEY
2229    GEORGE STOLTZ (STOLTZ LAND & CATTLE CO.)
1911                  COLDWATER PLANTING COMPANY
Name: EMPLOYER_NAME, dtype: object

5                               PROMAX INC.
11                    LOEWEN HARVESTING LLC
94                             69 FARMS LLC
106                            TURNER FARMS
36                              AGECY I LLC
46                 ALTENDORF TRANSPORT INC.
35                              IRISH FLATS
74     JIM AND ANN SHIPLEY WILLIAM SHIPLEY*
41                          K W ENTERPRISES
77                    MYRKA MIREYA CARDENAS
53                            AGECY II, LLC
3                     ANTON FERTILIZER INC.
105                             J & L FARMS
0                            J&J HARVESTING
19                       CISCO PRODUCE INC.
Name: Name, dtype: object

In [93]:
## insert your code for assigning the uppercase names back to the data
# done in part A

### 2.3.2 Clean up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [94]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [95]:
## insert your code here with the regex pattern for part A
# made into a function
def clean_name(name: str):
    pattern = r"(LLC|INC|CO)\."
    return re.sub(pattern, r"\1", name)


## insert your code to use re.sub to apply the pattern to the test cases for part B
print(pos_example_1 + " -->  " + clean_name(pos_example_1))
print(pos_example_2 + " -->  " + clean_name(pos_example_2))
print(neg_example + " -->  " + clean_name(neg_example))


CISCO PRODUCE INC. -->  CISCO PRODUCE INC
AVOYELLES HONEY CO., LLC -->  AVOYELLES HONEY CO, LLC
E.V. RANCH LLP -->  E.V. RANCH LLP


### 2.3.3 Clean employer names (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [96]:
## your code here to clean the columns

jobs["name_clean"] = [clean_name(name) for name in jobs["EMPLOYER_NAME"]]
debar_clean["name_clean"] = [clean_name(name) for name in debar_clean["Name"]]

In [97]:
## your code here to print the head
jobs[jobs["name_clean"] != jobs["EMPLOYER_NAME"]][["EMPLOYER_NAME", "name_clean"]].head(n=10)
debar_clean[debar_clean["name_clean"] != debar_clean["Name"]][["Name", "name_clean"]].head(n=10)

,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"
19,HAYS HARVESTING INC.,HAYS HARVESTING INC
34,"ELW FARMS, INC.","ELW FARMS, INC"
41,"CRUM FARMING, INC.","CRUM FARMING, INC"
45,"REYNA HARVESTING, INC.","REYNA HARVESTING, INC"
59,"BRUSHVALE SEED, INC.","BRUSHVALE SEED, INC"


,Name,name_clean
3,ANTON FERTILIZER INC.,ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC.","GREAT PLAINS FLUID SERVICE, INC"
5,PROMAX INC.,PROMAX INC
13,REIMER'S INC.,REIMER'S INC
19,CISCO PRODUCE INC.,CISCO PRODUCE INC
23,REIMER’S INC.,REIMER’S INC
33,GERONIMO SHEEP CO.,GERONIMO SHEEP CO
46,ALTENDORF TRANSPORT INC.,ALTENDORF TRANSPORT INC
56,CISCO PRODUCE INC.,CISCO PRODUCE INC
72,"SAXTONS RIVER ORCHARDS, INC.","SAXTONS RIVER ORCHARDS, INC"


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe. Write a sentence describing how name cleaning affected the match results.

**Note**: this manual cleaning process is inefficient and may miss other likely matches. A better approach would be fuzzy matching, which would recognize that 'Slash EV ranch' is a highly similar string to 'slash ev ranch llp' and match them without us needing to use regex to make the strings identical. We may talk about this in class later if time!

In [98]:
## your code here
# A. 
joined2 = pd.merge(jobs, debar_clean,
                    on = 'name_clean',
                    how='inner')
joined2

# B.
def clean_name2(name: str):
    return re.sub(r'(\.)|(\sLLP)', '', name)
# test on example
print("SLASH E.V. RANCH LLP" + " -->  " + clean_name2("SLASH E.V. RANCH LLP"))

debar_clean["name_clean_2"] = [clean_name2(name) for name in debar_clean["name_clean"]]

# C.
joined3 = pd.merge(jobs, debar_clean,
                   left_on = 'name_clean',
                   right_on  = "name_clean_2",
                   how='left', indicator = True)
joined3[joined3['_merge'] == 'both']
# Optimistically, we doubled our output. Realistically, we only found one new match and did quite poorly.

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,name_clean,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,City
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, FL",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol,FL,Sebring


SLASH E.V. RANCH LLP -->  SLASH EV RANCH


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,Violation,Duration,Start date,End date,v_number,State,City,name_clean_y,name_clean_2,_merge
791,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol,FL,Sebring,RAFAEL BARAJAS,RAFAEL BARAJAS,both
1115,H-300-20306-894148,Determination Issued - Certification,2020-11-02 18:11:29.140,2020-11-24 00:00:00.000,Individual Employer,N,Seasonal,N,SLASH EV RANCH,NaN,...,WHD Debarment,1 year,11/15/2014,11/14/2015,viol,CO,Rifle,SLASH E.V. RANCH LLP,SLASH EV RANCH,both


(your interpretation here)

# 3. Regex to separate companies from individuals (6 points)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [99]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern
def extractor(string: str):
    pattern = r'.+(?=\s\(COMPANY\))|(?<=\sAND\s).+(?=\s\(INDIVIDUAL\)\*)'
    matches = re.findall(pattern, string)
    return matches

## your code here to apply it to the pos_example
print(extractor(pos_example))
## your code here to apply it to the negative example
print(extractor(neg_example))

['COUNTY FAIR FARM', 'ANDREW WILLIAMSON']
[]


C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [100]:
# your code here
# initalize for testing purposes
debar_clean["co_name"] = debar_clean["name_clean"]
debar_clean["ind_name"] = debar_clean["name_clean"]


for name in debar_clean['name_clean']:
    if len(extractor(name)) > 0:
        
        debar_clean.loc[debar_clean['name_clean']==name, "co_name"] = extractor(name)[0]
        debar_clean.loc[debar_clean['name_clean']==name, "ind_name"] = extractor(name)[1]
debar_clean


,Name,"City, State",Violation,Duration,Start date,End date,v_number,State,City,name_clean,name_clean_2,co_name,ind_name
0,J&J HARVESTING,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,viol,ND,Leads,J&J HARVESTING,J&J HARVESTING,J&J HARVESTING,J&J HARVESTING
1,"STAHLMAN APIARIES, INC","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,viol,SD,Selby,"STAHLMAN APIARIES, INC","STAHLMAN APIARIES, INC","STAHLMAN APIARIES, INC","STAHLMAN APIARIES, INC"
2,TRUST NURSERY,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,viol,NY,Pulaski,TRUST NURSERY,TRUST NURSERY,TRUST NURSERY,TRUST NURSERY
3,ANTON FERTILIZER INC.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol,KS,Dighton,ANTON FERTILIZER INC,ANTON FERTILIZER INC,ANTON FERTILIZER INC,ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,viol,KS,Greensburg,"GREAT PLAINS FLUID SERVICE, INC","GREAT PLAINS FLUID SERVICE, INC","GREAT PLAINS FLUID SERVICE, INC","GREAT PLAINS FLUID SERVICE, INC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,WALKER PLACE,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,viol,IL,Danville,WALKER PLACE,WALKER PLACE,WALKER PLACE,WALKER PLACE
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,viol,ME,Jefferson,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON
110,JESUS LEDESMA,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,viol,FL,Mulberry,JESUS LEDESMA,JESUS LEDESMA,JESUS LEDESMA,JESUS LEDESMA
112,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),"Collins, GA",WHD Debarment,3 years,4/9/17,4/9/20,viol,GA,Collins,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)


   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [101]:
# your code here
print("Positive Example")
debar_clean[debar_clean.name_clean.str.contains("COMPANY")][['name_clean','co_name','ind_name','Violation']]
print("Negative Example")
debar_clean[debar_clean.name_clean.str.contains("CISCO")][['name_clean','co_name','ind_name','Violation']]


Positive Example


,name_clean,co_name,ind_name,Violation
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON,WHD Debarment


Negative Example


,name_clean,co_name,ind_name,Violation
19,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Failure to respond to audit (no response)
56,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Impeding the Audit Process – Non- Response


# 4. Optional extra credit: Geospatial visualization (2 points)

Geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on a map of Georgia. This involves Googling and using external sources to figure out the code (a common practice in real-life data science), since we haven't spatial data in the course. 

**Hints:**
- Relevant columns include `EMPLOYER_ADDRESS_1` 
- The geocoding might have a long runtime, so feel free to implement it in a separate `.py` script that you submit alongside your notebook and to just read in the geocoded data

**Resources:**
- [Discussion of geocoding addresses -> lat/long](https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/)
- [Discussion of plotting lat/long dots against a map using geopandas](https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73)

In [147]:
## your code here

# Geocode addresses
import geopy
import geopandas
from geopy.extra.rate_limiter import RateLimiter


data = jobs["EMPLOYER_ADDRESS_1"]
data.shape
# drop duplicates
data = data.drop_duplicates()
data.shape
#drop addresses with a note or PO box
data = data[~data.str.contains("Note|PO|P.O.|P O", regex=True)]
data += ", Georgia"
data.shape

subset = data[0:2]
subset 

(2720,)

(2161,)

(2054,)

0            8433 NE 13th Ave, Georgia
1    982 Northcutts Cove Road, Georgia
Name: EMPLOYER_ADDRESS_1, dtype: object

In [150]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded
import time


locator = Nominatim(user_agent='QSS20 - Dartmouth')
# 1 - conveneint function to delay between geocoding calls
# geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

def geocode_me(location):
    time.sleep(1)
    try:
        return locator.geocode(location)
    except (GeocoderTimedOut, GeocoderQuotaExceeded) as e:
        if GeocoderQuotaExceeded:
            print(e)
        else:
            print(f'Location not found: {e}')
            return None

# 2 - create location column
subset['location'] = subset.apply(lambda x: geocode_me(x))

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
subset['point'] = subset['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# 4 - split point column into latitude, longitude and altitude columns
# subset[['latitude', 'longitude', 'altitude']] = pd.DataFrame(subset['point'].tolist(), index=subset.index)

subset


0                                   8433 NE 13th Ave, Georgia
1                           982 Northcutts Cove Road, Georgia
location    0           None
1           None
location    ...
point       0           None
1           None
location    ...
Name: EMPLOYER_ADDRESS_1, dtype: object

In [142]:
subset

0            8433 NE 13th Ave, Georgia
1    982 Northcutts Cove Road, Georgia
2        7366 State Hwy 146 S, Georgia
3              4198 Road 2049, Georgia
4     400 Eagle Lake Loop Rd., Georgia
Name: EMPLOYER_ADDRESS_1, dtype: object